# 一、比赛介绍及重难点剖析
## 1.1 赛题背景
在众多大规模视频分析情景中，从冗长未经修剪的视频中定位并识别短时间内发生的人体动作成为一个备受关注的课题。当前针对人体动作检测的解决方案在大规模视频集上难以奏效，高效地处理大规模视频数据仍然是计算机视觉领域一个充满挑战的任务。其核心问题可以分为两部分，一是动作识别算法的复杂度仍旧较高，二是缺少能够产生更少视频提案数量的方法（更加关注短时动作本身的提案）。

这里所指的视频动作提案是指一些包含特定动作的候选视频片段。为了能够适应大规模视频分析任务，时序动作提案应该尽可能满足下面两个需求：
（1）更高的处理效率，例如可以设计出使时序视频片段编码和打分更高效的机制；
（2）更强的判别性能，例如可以准确定位动作发生的时间区间。

本次比赛旨在激发更多的开发者和研究人员关注并参与有关视频动作定位的研究，创建性能更出色的动作定位模型。[赛题连接](https://aistudio.baidu.com/aistudio/competition/detail/127/0/introduction)

## 1.2 重难点剖析
乒乓球与其他运动项目相比，其动作类型的区分难度更大，这对识别任务来说是极大的挑战。对于乒乓球动作定位任务，主要难点如下：

1. 乒乓球远动员的挥拍动作很难通过一帧或几帧图像中的人物姿态去定位动作发生的时间；
2. 乒乓球远动员某些小幅度挥拍动作和非挥拍动作的区别仅存于某几帧的细微差异，判别难度较高；
3. 乒乓球远动员的挥拍动作只需要很短时间，通常小于1秒，这对定位模型的敏感度要求级高；
4. 本次比赛所使用的数据输入为ppTSM抽取的视频特征（连续帧特征)，定位模型可使用的数据形式会受到约束；
5. 所有数据均采集自乒乓球转播画面，机位固定，角度为正对挥拍运动员。

# 二、数据集介绍及评价指标
## 2.1数据集介绍
本次比赛的数据集包含了19-21赛季兵乓球国际比赛（世界杯、世锦赛、亚锦赛，奥运会）和国内比赛（全运会，乒超联赛）中标准单机位高清转播画面的特征信息，共包含912条视频特征文件，每个视频时长在0～6分钟不等，特征维度为2048，以pkl格式保存。我们对特征数据中面朝镜头的运动员的回合内挥拍动作进行了标注，单个动作时常在0～2秒不等，训练数据为729条标注视频，而A榜测试集共91个未标注视频，B榜测试集共92个未标注视频，其中训练数据标签以json格式给出。更详细信息见赛题[数据集及基线](https://aistudio.baidu.com/aistudio/competition/detail/127/0/datasets)部分。

## 2.2 评价指标
本次比赛我们采用AR@AN曲线下面积来评判模型效果。
AR（Average Recall） 即平均召回率，是由不同的时序交并比tIoU下计算得到的召回均值，tIoU 以0.05的步长从0.5取至0.9（包含0.5和0.9）。
AN（Average Number of Proposal per Video） 即平均提案数量，由提案总数除以测试集中视频总数得到。在计算AR@AN曲线下面积AUC时，AN以1为步长从0到100变化。
我们将对提交的文档进行打分，打分规则如下:

![](https://ai-studio-static-online.cdn.bcebos.com/de5c53104d1f4074bc2b1d3222c629102f86f7765d0943718e7bfe63c6d6873c)
详见[赛题说明](https://aistudio.baidu.com/aistudio/competition/detail/127/0/task-definition)

## 2.3 数据处理方案
先解压训练集和测试集（A榜或B榜），对于训练集，我们按照9：1或者39：1划分，前者是新的训练集，后者是验证集。对于训练，我们按照[BMN](https://openaccess.thecvf.com/content_ICCV_2019/papers/Lin_BMN_Boundary-Matching_Network_for_Temporal_Action_Proposal_Generation_ICCV_2019_paper.pdf)论文单元3.5中Training Data Construction所讲，以一个滑动窗口对每一个长视频序列进行切分，其中步长为窗口长度的一半，实验中我们最终采取窗口大小为100，注意的是我们只保留包含至少一个完整提案的切片用于训练期间的模型训练与验证，保留的切片存放在/home/aistudio/data/pre_feat中。划分之后每个用于训练或验证的切片对应的标签等信息存放在/home/aistudio/PaddleVideo/data/bmn/annotations_win.json文件中，其中训练视频切片与验证视频切片用"subset"属性区分，"train"为训练视频切片，"validation"为验证视频切片。

除此以外，为了在训练完模型之后对保存的模型进行筛选得到其中较好的模型，我们先后设计了两种验证方案。第一种，就是基于以上提到的验证集视频切片划分，将每个切片当成一个完整视频，这样所得到的仅包含验证视频切片对应的标签等信息的json文件为/home/aistudio/PaddleVideo/data/bmn/annotations_val_gt.json。但是这样的一种验证方案是与测试集预测方案不符的。对于测试集，它是无标签的，所以按照相同的滑动窗口划分切片方式，它所保留的是所有的切片，即每个切片不一定包含提案，而且预测之后是要将每个视频对应的所有切片预测的提案进行汇总并进一步使用soft nms等方法筛选之后作为原未划分视频的提案信息。因此为了与预测方案保持一致，我们采取第二种验证方案，即我们对验证集视频采取相同的滑动窗口划分方式，但保留所有的切片，即每个切片不一定包含提案，最后将每个视频对应的所有切片预测的提案进行汇总筛选之后，基于验证集原未划分视频来进行评价指标AUC筛选出验证集上较好模型用于预测。其中第二种方案切片视频存放在/home/aistudio/data/val_feat，验证集原未划分视频对应标签信息保存在/home/aistudio/PaddleVideo/data/bmn/annotations_val_gt_merged.json。

对于测试集切片划分，与上一段所讲一致，即以一个滑动窗口对每一个长视频序列进行切分，其中步长为窗口长度的一半，实验中我们最终采取窗口大小为100，产生的所有切片保留，即每个切片不一定包含提案，存放在/home/aistudio/data/test_feat，切片信息存放在/home/aistudio/PaddleVideo/data/bmn/annotations_test.json。使用所有切片预测之后，要对预测提案进行后处理，即将每个视频对应的所有切片预测的提案进行汇总并进一步使用soft nms等方法筛选之后作为原未划分视频的提案信息，保存在submission.json中，打包之后即可提交A榜或B榜。

# 三、思路演进及最终方案
## 3.1 思路演进及A榜分数比较

这里我们从这两个月中我们对网络进行几十次修改的方案中拿出来几个典型的案例（与最终方案相关）。
### 3.1.1 基础方案 BMN网络
[BMN](https://openaccess.thecvf.com/content_ICCV_2019/papers/Lin_BMN_Boundary-Matching_Network_for_Temporal_Action_Proposal_Generation_ICCV_2019_paper.pdf)模型是百度自研，2019年ActivityNet夺冠方案，为视频动作定位问题中proposal的生成提供高效的解决方案，在PaddlePaddle上首次开源。此模型引入边界匹配(Boundary-Matching, BM)机制来评估proposal的置信度，按照proposal开始边界的位置及其长度将所有可能存在的proposal组合成一个二维的BM置信度图，图中每个点的数值代表其所对应的proposal的置信度分数。网络由三个模块组成，基础模块作为主干网络处理输入的特征序列，TEM模块预测每一个时序位置属于动作开始、动作结束的概率，PEM模块生成BM置信度图。
![](https://ai-studio-static-online.cdn.bcebos.com/46e06abd697843e7b4d0535ad37396bb3e01fcf7b3164ee4935109c915bd72cd)

![](https://ai-studio-static-online.cdn.bcebos.com/eae4bd48e2f548eaa670f9b98ce0d3db6fa64c4a65144c3592a6e8dfc1fb3b88)

#### 3.1.1.1 batch_size的选择

| batch size | A榜分数 | 
| -------- | -------- | 
| 16     | 46.595     | 
| 32     | 46.895     | 

其中bs=4, bs=8的结果分不清了，只列举bs=16,32的，至此所有模型训练基本使用bs=32

#### 3.1.1.2 D, T大小的选取

| D，T | A榜分数 | 
| -------- | -------- | 
| 100     | 46.895     | 
| 200    | 45.459     | 

其中D=100使用bs=32，D=200使用bs=8（再大会报错）。而且我们做了数据分析，发现D=60即可覆盖到99%的完整提案，因此之后均选择D=100（T==D）

也进行过学习率和优化器、衰减策略调整，不再赘述，之后所有实验基本按照原PaddleVideo的默认配置，除了把epochs调为20

#### 3.1.1.3 提案汇总之后的后处理

使用soft nms再对汇总的提案进行筛选可进一步提高分数。
| 使用soft nms | A榜分数 | 
| -------- | -------- | 
| 否     | 37.428     | 
| 是    | 45.459     | 

### 3.1.2 BMN网络+SE层

这里我们在BMN网络基础上在Base Module部分增加SE注意力机制。如下图所示：
![](https://ai-studio-static-online.cdn.bcebos.com/45a27d959f5147eab381ac4069c5058d6a0af70d0cdc410ebb4dec12dfe1f7b3)

| 模型 | A榜分数 | 
| -------- | -------- | 
| BMN     | 46.895     | 
| BMN+se     | 46.996     | 

### 3.1.3 BMN网络+SE层+Shuffle通道打乱

注意到BMN网络的Base Module的一维卷积使用的分组卷积，且连续使用两次，考虑到ShufffleNet，我们也使用通道Shuffle来使得不同组间交流信息。见下图：
![](https://ai-studio-static-online.cdn.bcebos.com/2334b86e3f314f608d91b8521bf913c97e3f13eb3da848d7b598fe9843367f24)

| 模型 | A榜分数 | 
| -------- | -------- | 
| BMN     | 46.895     | 
| BMN+se     | 46.996     |
| BMN+se+s     | 47.23     |

### 3.1.4 BMN网络+Shuffle通道打乱+Unet结构
将BMN网络的Base Module部分改为降维模块+Unet结构来提升模型表达能力，为了降低增加卷积数量带来的参数量和浮点数运算量，我们均使用分组卷积和1x1卷积，同时尽可能使用相对较少的通道数。模型细节见下图：
![](https://ai-studio-static-online.cdn.bcebos.com/22c2486df54e4226958f79c8bf84f621e438216f3f79481ea2a995fadd40ea2a)

![](https://ai-studio-static-online.cdn.bcebos.com/0ae8e985d3e2472fb77d9a4887e02dfe87d1fc8c1756425db315e5463341d398)

增加残差跳过连接res之后的Base Module，如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/3e6c37c889164438b492a558f88a68dce63a9348d31649fbb476db23ba685240)


| 模型 | A榜分数 | 
| -------- | -------- | 
| BMN     | 46.895     | 
| BMN+se     | 46.996     |
| BMN+se+s     | 47.23     |
| BMN+se+s+bunet     | 47.227     |
| BMN+s+bunet     | 47.649     |
| BMN+s+bunet+res     | 47.72     |

因此，我们又放弃了se注意力机制。

#### 注意：提案汇总之后的后处理所用的soft nms针对长视频序列做的一点改进。
原来：
```
               tmp_width = tend[max_index] - tstart[max_index]
               if tmp_iou > t1 + (t2 - t1) * tmp_width:
```
修改之后：
```
                 tmp_width = (tend[max_index] - tstart[max_index])/dscale 
                if tmp_iou > t1 + (t2 - t1) * tmp_width:
```
对长视频序列的提案间隔进行了归一化操作，dscale即视频切片持续时间，当然这不是最优的方案，大家可以进一步的超参搜索。
| 修改 | A榜分数 | 
| -------- | -------- | 
| 前     | 47.542     | 
| 后     | 48.123     |

因此之后均使用修改后的soft nms对测试视频提案进行后处理筛选。

#### 注意：以下方案均采用39：1的数据集划分，挑选较好模型同时利用验证损失曲线和2.3提到的使用AUC的第二种的验证方案，而以上均是采用9：1的数据集划分，仅参考验证损失曲线来挑选模型。

划分改为39：1的原因是，如果验证集视频数量多，使用AUC的第二种的验证方案验证所有模型权重的时间较长，而使用39:1减少验证视频数量，要的时间短了，但可靠性降低了，因此只能作为一种参考，所以大家自己可以权衡。

### 3.1.5 BMN网络+Shuffle通道打乱+Unet结构+位置编码
本来是打算增加位置编码之后，确定视频开始与结束的差异，从而将视频时间正向，反向两次传入模型的时域评估模块，利用类似集成的方式来提高提案开始点和结束点的预测精度，但考虑到两次可能增加一部分计算量，没有采用，但增加位置编码之后仍然显著提高了精度。
第一种增加位置编码的位置（仅对Base Module修改， 其他相同与BMN）：
![](https://ai-studio-static-online.cdn.bcebos.com/debb72b645e9454f8b588bce3303161dae2317cb1cfe48f8af9f63af3b4aac42)

第二种，即我们的最终方案：
![](https://ai-studio-static-online.cdn.bcebos.com/a2da861e2f044ecdb32747958517a321d8eecb68d72d464da61e53967d5ff4e5)

| 位置编码位置 | A榜分数 | 
| -------- | -------- | 
| 第一种     | 48.5126     | 
| 第二种     | 48.82818     |

## 3.2 最终方案

我们采用3.1.5 的第二种增加位置编码方式的与Unet结构结合使用Shuffle的BMN网络，结构如下所示：
![](https://ai-studio-static-online.cdn.bcebos.com/bedf06dcc5984c2193da208259efdaef536375977fd041f7a9db9690a1c6fabb)
![](https://ai-studio-static-online.cdn.bcebos.com/af22ec3c1aec49eba9474350c3421b3c0ca10b0161154525842e61848bd34b32)

数据处理方案见2.3，参数选择可以见3.1

# 四、基于PaddleVideo的模型训练
模型训练是基于PaddleVideo套件，并增加了generate_data_for_training.py和utils.py两个文件进行数据处理。除此之外，还对PaddleVideo套件做了略微的修改，以满足训练过程可视化等要求。

## 4.1 训练前的准备工作

### 4.1.1 训练数据解压及处理

In [2]:
%cd /home/aistudio/data/
%mkdir data122998/Features_competition_train
!tar -xf data122998/Features_competition_train.tar.gz -C /home/aistudio/data/data122998/Features_competition_train --strip-components 1 && rm -rf data122998/Features_competition_train.tar.gz
%cd /home/aistudio/

/home/aistudio/data
/home/aistudio


In [1]:
!python /home/aistudio/generate_data_for_training.py

Namespace(feat_root='/home/aistudio/data/data122998/Features_competition_train', orig_val_gt_json_path='/home/aistudio/PaddleVideo/data/bmn/annotations_val_gt_merged.json', train_feat_path='/home/aistudio/data/pre_feat', train_json_path='/home/aistudio/PaddleVideo/data/bmn/annotations_win.json', val_feat_path='/home/aistudio/data/val_feat', val_gt_json_path='/home/aistudio/PaddleVideo/data/bmn/annotations_val_gt.json', val_json_path='/home/aistudio/PaddleVideo/data/bmn/annotations_val_win.json', val_ratio=0.025, video_info_path='/home/aistudio/data/data122998/label_cls14_train.json', win_T=100)
准备划分并筛选得到用于训练验证的视频切片数据...
总共729条视频，其中710条视频划分后用于训练，其余19条视频用于验证！
开始以窗口大小为100，步长为50来划分每一条视频（保存的每个视频切片至少包含一个完整的提案）...
被丢弃的无效数据： {'url': 'bff88e62f7934de7b2685c4f776b80ce.mp4', 'total_frames': 4613, 'actions': []}
/home/aistudio/data/pre_feat中共有19595条切片数据！
准备划分得到训练完用于验证模型AUC的视频切片数据...
开始以窗口大小为100，步长为50来划分每一条视频（保存的每个视频切片不必包含提案）...
100%|███████████████████████████████████████████| 19/19 [00:06<00:00, 

### 4.1.2 安装更新依赖库

In [ ]:
%cd /home/aistudio/PaddleVideo
!python3.7 -m pip install --upgrade pip
!python3.7 -m pip install --upgrade -r requirements.txt
%cd /home/aistudio/

## 4.2 训练
先设置随机数种子，再启动训练命令。注意由于每次启动的机器不同，即使设置了随机数种子，但底层cudnn中卷积算法的不确定性仍然存在，会导致每次的训练结果有波动。

In [2]:
# 设置随机数种子
import sys
sys.path.append("/home/aistudio/")
from utils import set_seed_paddle, merging_output_per_video, checking_submission
set_seed_paddle(1024)

In [ ]:
# 训练
%cd /home/aistudio/PaddleVideo
#!python -B main.py -c configs/localization/my_yaml/bmn.yaml
!python -B main.py  --validate -c configs/localization/my_yaml/bmn.yaml
%cd /home/aistudio/

## 4.3 验证
训练之后得到一组模型权重，通过验证即可筛选出哪些模型权重在验证集上表现较好。如2.3数据处理方案所说，这里存在两种验证方案，但第二种模拟了预测方案，相对更为可靠。

### 4.3.1 采用第一种验证方案

In [ ]:
# 验证
%cd /home/aistudio/PaddleVideo
!python main.py --eval_model -c configs/localization/my_yaml/bmn_val.yaml -w output/BMN -o DATASET.test_batch_size=1
%cd /home/aistudio/

### 4.3.2 采用第二种验证方案（更推荐）

In [ ]:
# 验证
%cd /home/aistudio/PaddleVideo
!python main.py --eval_model -c configs/localization/my_yaml/bmn_val_merge.yaml -w output/BMN/ -o DATASET.test_batch_size=1
%cd /home/aistudio/

## 4.4 测试

### 4.4.1 解压并划分测试集

In [4]:
%cd /home/aistudio/data/
%mkdir data123009/Features_competition_test_B
!tar -xf data123009/Features_competition_test_B.tar.gz -C /home/aistudio/data/data123009/Features_competition_test_B --strip-components 1 && rm -rf data123009/Features_competition_test_B.tar.gz
%cd /home/aistudio/

/home/aistudio/data
mkdir: 无法创建目录"data123009/Features_competition_test_B": 文件已存在
/home/aistudio


In [6]:
# 划分测试视频
!python /home/aistudio/generate_data_for_testing.py

Namespace(feat_root='/home/aistudio/data/data123009/Features_competition_test_B', fps=25, test_feat_path='/home/aistudio/data/test_feat', test_json_path='/home/aistudio/PaddleVideo/data/bmn/annotations_test.json', win_T=100)
准备划分得到训练完用于验证模型AUC的视频切片数据...
开始以窗口大小为100，步长为50来划分每一条视频（保存的每个视频切片不必包含提案）...
100%|███████████████████████████████████████████| 92/92 [00:36<00:00,  2.50it/s]
/home/aistudio/data/test_feat中共有15815条切片数据！


### 4.4.2 预测
我们对PaddleVideo修改从而可以利用以下命令获得测试集视频切片的提案

In [7]:
# 预测 #test A有16102条切片数据，而test B有15815条
%cd /home/aistudio/PaddleVideo
!python main.py --test -c configs/localization/my_yaml/bmn_test.yaml -w output/BMN_best/BMN_epoch_00008.pdparams -o DATASET.test_batch_size=1
%cd /home/aistudio/

/home/aistudio/PaddleVideo
[03/03 22:42:44] DALI is not installed, you can improve performance if use DALI
[03/03 22:42:45] DATASET : 
[03/03 22:42:45]     batch_size : 32
[03/03 22:42:45]     num_workers : 8
[03/03 22:42:45]     test : 
[03/03 22:42:45]         file_path : data/bmn/annotations_test.json
[03/03 22:42:45]         format : BMNDataset
[03/03 22:42:45]         subset : validation
[03/03 22:42:45]         test_mode : predict
[03/03 22:42:45]     test_batch_size : 1
[03/03 22:42:45]     train : 
[03/03 22:42:45]         file_path : data/bmn/annotations_win.json
[03/03 22:42:45]         format : BMNDataset
[03/03 22:42:45]         subset : train
[03/03 22:42:45]     valid : 
[03/03 22:42:45]         file_path : data/bmn/annotations_win.json
[03/03 22:42:45]         format : BMNDataset
[03/03 22:42:45]         subset : validation
[03/03 22:42:45] ------------------------------------------------------------
[03/03 22:42:45] INFERENCE : 
[03/03 22:42:45]     dscale : 100
[03/03 

### 4.4.3 预测提案后处理
见2.3数据处理方案详细介绍

In [4]:
#合并切片视频的提案信息并逐源未切分视频再次进行soft nms来分别筛选得到100个提案用于提交
merging_output_per_video(test_root="/home/aistudio/data/data123009/Features_competition_test_B",  #未划分的原视频特征存放路径
                        proposal_filename='PaddleVideo/data/bmn/BMN_Test_results/bmn_results_validation.json', #预测的待拼接回每个原视频的切片提案信息文件路径
                        merging_output_filename="submission_B.json", #合并后每个视频对应的预测提案信息，用于提交
                        win_T=100, 
                        fps=25,
                        snms_alpha=0.4, 
                        snms_t1=0.55,
                        snms_t2=0.9)

100%|██████████| 92/92 [01:41<00:00,  1.10s/it]


### 4.4.5 核查结果与提交结果是否一致

In [5]:
#核查输出的submission.json与我们A榜或B榜提交的是否一致
checking_submission(json_path1="submission_B.json", json_path2="submission_A_B/submission_B.json")

两份提交文件完全一致!


# 五、总结及参考文献
## 5.1 总结

本次比赛看了最近几年顶会一部分的动作提案生成方面论文，对这个领域有了初步的认识，在训练调优和修改模型过程中，验证了许多想法，收获满满，感谢百度飞桨举办这次比赛！

## 5.2 参考文献

1. Lin T, Liu X, Li X, et al. BMN: Boundary-Matching Network for Temporal Action Proposal Generation[J]. arXiv preprint arXiv:1907.09702, 2019.
2. Su H, Gan W, Wu W, et al. Bsn++: Complementary boundary regressor with scale-balanced relation modeling for temporal action proposal generation[J]. arXiv preprint arXiv:2009.07641, 2020.
3. Tan J, Tang J, Wang L, et al. Relaxed transformer decoders for direct action proposal generation[C]//Proceedings of the IEEE/CVF International Conference on Computer Vision. 2021: 13526-13535.
4. Qing Z, Su H, Gan W, et al. Temporal context aggregation network for temporal action proposal refinement[C]//Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2021: 485-494.
5. Lin T, Zhao X, Su H, et al. Bsn: Boundary sensitive network for temporal action proposal generation[C]//Proceedings of the European conference on computer vision (ECCV). 2018: 3-19.
6. Gao J, Shi Z, Wang G, et al. Accurate temporal action proposal generation with relation-aware pyramid network[C]//Proceedings of the AAAI Conference on Artificial Intelligence. 2020, 34(07): 10810-10817.
